In [2]:
import numpy as np

In [ ]:
#gcs einheinte gcms einheiten because is cassical for electromagnetism

d_cm = 1

def iron_sphere_gravity_force(d_cm, g=981, rho=7.87):  #rho in g/cm^3, g in cm/s^2

    r = d_cm / 2
    V = (4/3) * np.pi * r**3
    mass = rho * V
    F = mass * g
    return mass, F

# example call
mass, F_g = iron_sphere_gravity_force(d_cm)

print(round(mass,2),'mass in g')
print(round(F_g,2),'force in g*cm/s^2')


4.12 mass in g
4042.43 force in g*cm/s^2


# Magnetic force

In [ ]:
Ms = 217.6*10000 #went to cm 
# general: 1 emg/g = 1 A m^2/kg = J/(T kg)    #https://royalsocietypublishing.org/doi/10.1098/rspa.1971.0044


def dipole_moment_iron(d_cm):
    # Iron constants
    # Sphere volume
    V = (4/3) * np.pi * (d_cm/2)**3  #cm
    # Magnetic dipole moment
    m = Ms * V
    return m   #A/cm



m_iron = dipole_moment_iron(d_cm)
print(f"Iron sphere dipole moment: {m_iron:.3e} A·cm²")
print(f"Iron sphere dipole moment: {m_iron/1000:.3e} memu")  #72 memu at 30 mT   checked

def necessary_gradient(m, F_g):     #A/cm , g*cm/s^2     
    #1mT = g/(s^2*A)
    dB_dz = F_g / m   
    return dB_dz   #mT/cm

dB_dz = necessary_gradient(m_iron, F_g)
print(f"Necessary magnetic field gradient: {dB_dz:.3e} mT/cm")
print(f"Necessary magnetic field gradient: {dB_dz*10:.3e} mT/m")   #up to 350 mT/m achieved by Navion 30mT




Iron sphere dipole moment: 1.139e+06 A·cm²
Iron sphere dipole moment: 1.139e+03 memu
Necessary magnetic field gradient: 3.548e-03 mT/cm
Necessary magnetic field gradient: 3.548e-02 mT/m


# necessary magnetic field gradient for my project is 4 orders of magnitude smaller than what navion can achieve!
# lets see what magnetic field we can generate with simple coils!

control unitis wiredes gcs units ohne mu! und Iabad
get rid of assumtion that iron is fully saturated!

In [ ]:
N = 100
R_cm = 2  #cm
z_cm = 5 #cm 
R_arduino = 10 #ohm
V = 3.3         # volts

#Why? In Gaussian units, the permeability of vacuum is dimensionless and equal to 1: 
#𝜇0=1μ0=1.Current must be in abA, lengths in cm, B in Gauss.

def B_axis_loop_cgs_voltage(N ,V, R_cm, z_cm, R_arduino):  #assumes voltage = Strom so no inductance!!!
    """
    Magnetic field along the axis (z) of a circular loop in CGS units,
    using applied voltage and loop resistance.

    Parameters:
    V : float
        Applied voltage in volts
    R_cm : float
        Loop radius in cm
    z_cm : float or ndarray
        Distance along the axis from the center in cm
    

    Returns:
    B : float or ndarray
        Magnetic flux density along the axis in Gauss (G)
    """
    # Convert voltage to current (assuming simple resistor)
    I_A= V / R_arduino          #  möglichst klein, weil ich grßen strom will
    I_abA = I_A * 0.1             # Convert A → abA for CGS

    # Magnetic field in Gauss
    B_G = N *(2 * np.pi * I_abA * R_cm**2) / ((z_cm**2 + R_cm**2)**(3/2))
    return B_G


B = B_axis_loop_cgs_voltage(N, V, R_cm, z_cm, R_arduino)  #in Gauss
print(f"Magnetic field: {B:.3f} G")
print(f"Magnetic field: {B/1000:.6f} mT")  #should be less than 30 mT becaues navion is strong!


Magnetic field: 0.531 G
Magnetic field: 0.000531 mT


In [46]:
def dBdz_axis_loop_cgs(V, R_cm, z_cm):
    """
    Magnetic field gradient along the axis of a circular loop in CGS units.

    Parameters:
    I_abA : float
        Current in abampere (CGS)
    R_cm : float
        Loop radius in cm
    z_cm : float
        Distance along the axis in cm

    Returns:
    dBdz : float
        Magnetic field gradient in Gauss/cm
    """
    I_A = V / R_arduino          # Current in Amperes
    I_abA = I_A * 0.1

    numerator = N*(-6) * np.pi * I_abA * R_cm**2 * z_cm
    denominator = (z_cm**2 + R_cm**2)**(5/2)
    return numerator / denominator



grad = dBdz_axis_loop_cgs(V, R_cm, z_cm)
print(f"dB/dz = {grad:.3e} G/cm")  #
print(f"dB/dz = {grad*10:.3e} mT/m") #no unit tested! to move against gravity!


print(f"Necessary magnetic field gradient: {dB_dz*10:.3e} mT/m")   #against gravity!




dB/dz = -2.747e-01 G/cm
dB/dz = -2.747e+00 mT/m
Necessary magnetic field gradient: 3.548e-02 mT/m


# usb c cable liefert 1.2 A